In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

from tqdm.notebook import tqdm

In [2]:
RANDOM_STATE = 20180724

n_splits = 4

In [3]:
path = "E:\datasets\[programmers]\challenges_849"

# Read Datasets
## Read train

In [4]:
df_train = pd.read_csv(os.path.join(path, "train.csv"))
print(df_train.shape)
df_train.head(5)

(6000, 3)


,userID,jobID,applied
0,fe292163d06253b716e9a0099b42031d,15de21c670ae7c3f6f3f1f37029303c9,0
1,6377fa90618fae77571e8dc90d98d409,55b37c5c270e5d84c793e486d798c01d,0
2,8ec0888a5b04139be0dfe942c7eb4199,0fcbc61acd0479dc77e3cccc0f5ffca7,1
3,f862b39f767d3a1991bdeb2ea1401c9c,3b5dca501ee1e6d8cd7b905f4e1bf723,0
4,cac14930c65d72c16efac2c51a6b7f71,287e03db1d99e0ec2edb90d079e142f3,0


## Read test

In [5]:
df_test = pd.read_csv(os.path.join(path, "test_job.csv"))
print(df_test.shape)
df_test.head(5)

(2435, 2)


,userID,jobID
0,ebaee1af0c501f22ddfe242fc16dae53,352407221afb776e3143e8a1a0577885
1,9ab05403ac7808cbfba3da26665f7a9c,96b9bff013acedfb1d140579e2fbeb63
2,33349e909eba71677299d2fc97e158b7,58d4d1e7b1e97b258c9ed0b37e02d087
3,ac985a9db5faeb44c94a334430ccc241,ccb0989662211f61edae2e26d58ea92f
4,d41e0e6f6f1e29098d9d152511503ab2,4a213d37242bdcad8e7300e202e7caa4


## company

In [6]:
df_company = pd.read_csv(os.path.join(path, "job_companies.csv"))
print(df_company.shape)
df_company.head(5)

(733, 3)


,companyID,jobID,companySize
0,00411460f7c92d2124a67ea0f4cb5f85,e5f6ad6ce374177eef023bf5d0c018b6,NaN
1,1905aedab9bf2477edc068a355bba31a,185e65bc40581880c4f2c82958de8cfe,11-50
2,89fcd07f20b6785b92134bd6c1d0fa42,0537fb40a68c18da59a35c2bfe1ca554,101-200
3,acf4b89d3d503d8252c9c4ba75ddbf6d,b7ee6f5f9aa5cd17ca1aea43ce848496,1-10
4,fb7b9ffa5462084c5f4e7e85a093e6d7,efe937780e95574250dabe07151bdc23,NaN


## jobtag

In [7]:
df_jobtag = pd.read_csv(os.path.join(path, "job_tags.csv"))
print(df_jobtag.shape)
df_jobtag.head(5)

(3477, 2)


,jobID,tagID
0,320722549d1751cf3f247855f937b982,d38901788c533e8286cb6400b40b386d
1,e744f91c29ec99f0e662c9177946c627,3948ead63a9f2944218de038d8934305
2,e744f91c29ec99f0e662c9177946c627,0e095e054ee94774d6a496099eb1cf6a
3,e820a45f1dfc7b95282d10b6087e11c0,7d771e0e8f3633ab54856925ecdefc5d
4,e820a45f1dfc7b95282d10b6087e11c0,6c8dba7d0df1c4a79dd07646be9a26c8


## tag

In [8]:
df_tag = pd.read_csv(os.path.join(path, "tags.csv"))
print(df_tag.shape)
df_tag.head(5)

(887, 2)


,tagID,keyword
0,602d1305678a8d5fdb372271e980da6a,Amazon Web Services(AWS)
1,e3251075554389fe91d17a794861d47b,Tensorflow
2,a1d50185e7426cbb0acad1e6ca74b9aa,Docker
3,884d79963bd8bc0ae9b13a1aa71add73,Git
4,4122cb13c7a474c1976c9706ae36521d,Python


## usertag

In [9]:
df_usertag = pd.read_csv(os.path.join(path, "user_tags.csv"))
print(df_usertag.shape)
df_usertag.head(5)

(17194, 2)


,userID,tagID
0,e576423831e043f7928d9ac113abbe6f,82c2559140b95ccda9c6ca4a8b981f1e
1,e576423831e043f7928d9ac113abbe6f,2ba8698b79439589fdd2b0f7218d8b07
2,e576423831e043f7928d9ac113abbe6f,351b33587c5fdd93bd42ef7ac9995a28
3,e576423831e043f7928d9ac113abbe6f,81e5f81db77c596492e6f1a5a792ed53
4,e576423831e043f7928d9ac113abbe6f,0e095e054ee94774d6a496099eb1cf6a


## submission

In [10]:
df_submission = pd.read_csv(os.path.join(path, "sample_output_job.csv"))
print(df_submission.shape)
df_submission.head(8)

(8, 1)


,applied
0,0
1,1
2,0
3,1
4,0
5,1
6,0
7,1


In [11]:
df_company.groupby('jobID')['companyID'].value_counts()

jobID                             companyID                       
00ac8ed3b4327bdd4ebbebcb2ba10a00  2dace78f80bc92e6d7493423d729448e    1
00e26af6ac3b1c1c49d7c3d79c60d000  c058f544c737782deacefa532d9add4c    1
00ec53c4682d36f5c4359f4ae7bd7ba1  8757150decbd89b0f5442ca3db4d0e0e    1
013a006f03dbc5392effeb8f18fda755  069d3bb002acd8d7dd095917f9efe4cb    1
0188e8b8b014829e2fa0f430f0a95961  07c5807d0d927dcd0980f86024e5208b    1
                                                                     ..
fed33392d3a48aa149a87a38b875ba4a  512c5cad6c37edb98ae91c8a76c3a291    1
ff4d5fbbafdf976cfdc032e3bde78de5  08419be897405321542838d77f855226    1
ffd52f3c7e12435a724a8f30fddadd9c  4f6ffe13a5d75b2d6a3923922b3922e5    1
ffeabd223de0d4eacb9a3e6e53e5448d  a8e864d04c95572d1aece099af852d0a    1
ffeed84c7cb1ae7bf4ec4bd78275bb98  c0f168ce8900fa56e57789e2a2f2c9d0    1
Name: companyID, Length: 733, dtype: int64

# Data Preprocessing

In [12]:
tags = df_tag['tagID'].unique()
tag_dict = dict()
for i, tag in enumerate(tags):
    tag_dict[tag] = i 

In [13]:
users = df_usertag['userID'].unique()
user_dict = dict()
for i, user in enumerate(users):
    user_dict[user] = i 

In [14]:
jobs = df_jobtag['jobID'].unique()
job_dict = dict()
for i, tag in enumerate(jobs):
    job_dict[tag] = i 

## One Hot Encoded
### Set Tag to One-Hot Encoding Vector

In [15]:
A = np.eye(len(tag_dict))
tag_onehot_dict = dict()
for key, val in tag_dict.items():
    tag_onehot_dict[key] = A[val,:]

### User Technical Stack. Add Tech. Tag Vector

In [16]:
user_stack = dict()

for i in range(df_usertag.shape[0]):
    userID = df_usertag.iloc[i]['userID']
    tagID = df_usertag.iloc[i]['tagID']
    if userID in user_stack.keys():
        user_stack[userID] += tag_onehot_dict[tagID]
    else:
        user_stack[userID] = tag_onehot_dict[tagID].copy()

### Check Stacks

In [17]:
sample = np.random.choice(len(user_stack), 10, replace = False)

for i, (key, val) in enumerate(user_stack.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
cac14930c65d72c16efac2c51a6b7f71 [0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 5. 0. 0. 0. 0. 5. 0. 0. 5. 0. 0. 0. 0. 0.
 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 5. 0. 5. 0. 0. 0. 5. 0.
 0. 0. 0. 5. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


기술 스택은 0, 1로만 이루어진 vector를 예상했지만, 실제로는 1보다 큰 값들이 여러개 포함되어 있다.
### User별 스택 확인

In [18]:
df_usertag.loc[df_usertag['userID'] == 'f69054686ba46877b6397ccdb8f51762', 'tagID'].value_counts()

20d135f0f28185b84a4cf7aa51f29500    26
7bccfde7714a1ebadf06c5f4cea752c1    26
aace49c7d80767cffec0e513ae886df0    26
f47330643ae134ca204bf6b2481fec47    26
9996535e07258a7bbfd8b132435c5962    26
4122cb13c7a474c1976c9706ae36521d    26
2ba8698b79439589fdd2b0f7218d8b07    26
81dc9bdb52d04dc20036dbd8313ed055    26
a8c88a0055f636e4a163a5e3d16adab7    26
c8ba76c279269b1c6bc8a07e38e78fa4    26
dd77279f7d325eec933f05b1672f6a1f    21
1e6e0a04d20f50967c64dac2d639a577    21
9bf31c7ff062936a96d3c8bd1f8f2ff3    21
e034fb6b66aacc1d48f445ddfb08da98    21
4c27cea8526af8cfee3be5e183ac9605    21
66808e327dc79d135ba18e051673d906    21
db2b4182156b2f1f817860ac9f409ad7    21
c9f95a0a5af052bffce5c89917335f67    21
4e2545f819e67f0615003dd7e04a6087    21
26588e932c7ccfa1df309280702fe1b5    21
c20ad4d76fe97759aa27a0c99bff6710    21
82c2559140b95ccda9c6ca4a8b981f1e    21
8a3363abe792db2d8761d6403605aeb7     5
81e5f81db77c596492e6f1a5a792ed53     5
7810ccd41bf26faaa2c4e1f20db70a71     5
e702e51da2c0f5be4dd354bb3

실제로도 한 유저에게 중복되는 `tagID`가 존재하는 것을 확인. 하지만 동일한 `tagID`가 여럿 있다는 사실 자체는 중요하지 않기 때문에, 기술 스택 유무만 저장 하도록 `user_stack`을 정의하자

In [19]:
for key, val in user_stack.items():
    user_stack[key] = (val > 0).astype(float)

In [20]:
sample = np.random.choice(len(user_stack), 10, replace = False)

for i, (key, val) in enumerate(user_stack.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
3203e06c1efc6fdb23016650d1d1888c [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


### Job Dict

In [21]:
job_dict = dict()

for i in range(df_jobtag.shape[0]):
    jobID = df_jobtag.iloc[i]['jobID']
    tagID = df_jobtag.iloc[i]['tagID']
    if jobID in job_dict.keys():
        job_dict[jobID] += tag_onehot_dict[tagID]
    else:
        job_dict[jobID] = tag_onehot_dict[tagID].copy()

In [22]:
sample = np.random.choice(len(job_dict), 10, replace = False)

for i, (key, val) in enumerate(job_dict.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
7cce53cf90577442771720a370c3c723 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


In [23]:
def DataFrame2Array(df):
    X = []
    y = []
    for i in range(df.shape[0]):
        userID = df.iloc[i]['userID']
        jobID = df.iloc[i]['jobID']
        
        if 'applied' in df.columns:
            apply = df.iloc[i]['applied']
        else:
            apply = []
        X.append(np.concatenate([user_stack[userID], job_dict[jobID]]).reshape(1, -1))
        y.append(np.array(apply).reshape(1,-1))

    X = np.concatenate(X)
    y = np.concatenate(y)
    
    return X, y

X, y = DataFrame2Array(df_train)
X_test, _ = DataFrame2Array(df_test)

## Data Loader
### First, tarin_val_split

In [24]:
kf = KFold(n_splits= n_splits)

In [25]:
ESTIMATORS = [50]
DEPTH = [50]
SAMPLE_SPLIT = [10]
SAMPLE_LEAF = [1]

estimators, max_depths, sam_splits, sam_leaves = np.meshgrid(range(len(ESTIMATORS)), range(len(DEPTH)), range(len(SAMPLE_SPLIT)), range(len(SAMPLE_LEAF)))

estimators = estimators.reshape(-1,1)
max_depths = max_depths.reshape(-1,1)
sam_splits = sam_splits.reshape(-1,1)
sam_leaves = sam_leaves.reshape(-1,1)

In [26]:
score_list = []
for i in tqdm(range(estimators.shape[0])):
    n_estimators = ESTIMATORS[int(estimators[i])]
    max_depth = DEPTH[int(max_depths[i])]
    min_samples_split = SAMPLE_SPLIT[int(sam_splits[i])]
    min_samples_leaf = SAMPLE_LEAF[int(sam_leaves[i])]

    sum_score = 0
    for train_index, valid_index in tqdm(kf.split(X)):
        model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, 
                                       random_state = RANDOM_STATE, verbose = False)
        model.fit(X[train_index, :], y[train_index].reshape(-1))
        sum_score += model.score(X[valid_index, :], y[valid_index].reshape(-1))

    score = sum_score / n_splits
    score_list.append(score)
    print(f"{n_estimators}, {max_depth}, {min_samples_split}, {min_samples_leaf} = {score}")

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

50, 50, 10, 1 = 0.8636666666666667


In [27]:
i = np.array(score_list).argmax()

n_estimators = ESTIMATORS[int(estimators[i])]
max_depth = DEPTH[int(max_depths[i])]
min_samples_split = SAMPLE_SPLIT[int(sam_splits[i])]
min_samples_leaf = SAMPLE_LEAF[int(sam_leaves[i])]

print(f"max score - {n_estimators}, {max_depth}, {min_samples_split}, {min_samples_leaf} = {np.array(score_list).max()}")

max score - 50, 50, 10, 1 = 0.8636666666666667


In [28]:
model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, 
                               random_state = RANDOM_STATE, verbose = False)
model.fit(X, y.reshape(-1))

RandomForestClassifier(max_depth=50, min_samples_split=10, n_estimators=50,
                       random_state=20180724, verbose=False)

In [29]:
df_tag

,tagID,keyword
0,602d1305678a8d5fdb372271e980da6a,Amazon Web Services(AWS)
1,e3251075554389fe91d17a794861d47b,Tensorflow
2,a1d50185e7426cbb0acad1e6ca74b9aa,Docker
3,884d79963bd8bc0ae9b13a1aa71add73,Git
4,4122cb13c7a474c1976c9706ae36521d,Python
...,...,...
882,818f4654ed39a1c147d1e51a00ffb4cb,활용
883,7cce53cf90577442771720a370c3c723,mybais
884,c2aee86157b4a40b78132f1e71a9e6f1,*
885,1fb2a1c37b18aa4611c3949d6148d0f8,Autodesk Maya


In [30]:
user_stack_importance = model.feature_importances_[:int(1774/2)]
jobs_stack_importance = model.feature_importances_[int(1774/2):]

df_tag['user_importance'] = user_stack_importance
df_tag['jobs_importance'] = jobs_stack_importance

In [31]:
df_tag.sort_values('user_importance', ascending=False).head()

,tagID,keyword,user_importance,jobs_importance
58,11c484ea9305ea4c7bb6b2e6d570d466,Google AI Hub,0.011136,0.000000
35,9adeb82fffb5444e81fa0ce8ad8afe7a,Linux,0.009865,0.008772
15,f47330643ae134ca204bf6b2481fec47,Java,0.008371,0.010676
95,a42a596fc71e17828440030074d15e74,Laravel,0.008118,0.000431
347,1368ba1ab6ed38bb1f26f36673739d54,Azure Dedicated HSM,0.007841,0.000000


In [32]:
df_tag.sort_values('jobs_importance', ascending=False).head()

,tagID,keyword,user_importance,jobs_importance
222,93fb9d4b16aa750c7475b6d601c35c2c,JIRA,0.001568,0.016342
10,0e095e054ee94774d6a496099eb1cf6a,JavaScript,0.005455,0.012845
42,2ba8698b79439589fdd2b0f7218d8b07,Spring,0.004434,0.011848
4,4122cb13c7a474c1976c9706ae36521d,Python,0.006737,0.011432
34,3948ead63a9f2944218de038d8934305,Node.js,0.005414,0.011422


In [33]:
pred = model.predict(X_test)
pred.shape
# submission = pd.DataFrame(pred.reshape(-1, 1), columns = 'applied')
# print(submission.shape)
# submission.head()

(2435,)

In [35]:
submission = pd.DataFrame([])
submission['applied'] = pred

print(submission.shape)
print(submission.mean())
print(submission.dtypes)
submission.sample(10)

(2435, 1)
applied    0.019302
dtype: float64
applied    int64
dtype: object


,applied
519,0
1487,0
1064,0
975,0
372,0
683,0
466,0
179,0
384,0
1557,0


In [ ]:
submission.to_csv("submission.csv", index = False)